In [1]:
# Homework 5
# Mass Distribution of Each Galaxy
# Justin Ugaitafa

# import modules
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib

from ReadFile import Read
from CenterOfMass import CenterOfMass
from astropy.constants import G # imports gravitaional constant

In [2]:
# converting gravitational constant to units of kpc*km^2/s^2/Msun
G = G.to(u.kpc*u.km**2/u.s**2/u.Msun) # 4.498768e-6*u.kpc**3/u.Gyr**2/u.Msun

# Class to define the mass distribution of a given galaxy and snapshot
class MassProfile:

    def __init__(self, galaxy, snap):
    # Initialize the instance of this class with the following properties:

        # add a string of the filenumber to the value "000"
        ilbl = '000' + str(snap)
        # remove all but last 3 digits
        ilbl = ilbl[-3:]
        self.File = '%s_'%(galaxy) + ilbl + '.txt'

        # reads data in given file using Read
        self.time, self.total, self.data = Read(self.File)
    
        # store the mass, positions, velocities of only the particles of the
        # given type into arrays
        self.m = self.data['m']
        self.x = self.data['x']*u.kpc
        self.y = self.data['y']*u.kpc
        self.z = self.data['z']*u.kpc

        self.gname = galaxy # store galaxy name

    def MassEnclosed(self, PType, r):
    # Function that will compute the mass within any given radius with respect
    # to the COM for a specified galaxy and specified component of that galaxy.
    # input: the array of all the radii of the total mass enclosed
    # return: the array of the mass of each particle within each specified
    #         radius.
        # read in COM position from last HW
        COM = CenterOfMass(self.File,2)
        Position = COM.COM_P(0.1)

        #creates an array to store indexes of particles of desired Ptype
        index = np.where(self.data['type'] == PType)
        
        # 1) find the x,y,z positions of each indicidual particle within
        #    specified radius.
        xG = self.x[index] - Position[0]
        yG = self.y[index] - Position[1]
        zG = self.z[index] - Position[2]
        # 2) find the mass of each particle
        mG = self.m[index]
        # computes magnitude of position vector
        rG = np.sqrt(xG**2 + yG**2 + zG**2)

        # Array to store enclosed mass as a function of the input radius array
        Menc = np.zeros(np.size(r))
        
        # for loop to iterate over the radius given at each array element
        for i in range(np.size(r)):
            # Only want particles within the given radius
            indexR = np.where(rG < r[i]*u.kpc)
            Menc[i] = np.sum(mG[indexR])*1e10 # total mass of particles

        return Menc*u.Msun
    
    def MassEnclosedTotal(self, r):
    # Function to compute mass enclosed within an enclosed radius
    # input: array(x,y,z) of radii
    # return: array of masses (Msun) representing total mass(bulge+disk+halo) at
    #         each radius of the input array
        EncHaloM = self.MassEnclosed(1,r) # Mass of particles enclsoed in halo
        EncDiskM = self.MassEnclosed(2,r) # Mass of particles enclosed in disk
        EncBulgeM = self.MassEnclosed(3,r) # Mass of particles enclosed in bulge

        totalM = EncHaloM + EncDiskM + EncBulgeM

        if(self.gname == 'M33'): # M33 has no bulge
            totalM  = EncHaloM + EncDiskM

        return totalM

    def HernquistMass(self, r, a, Mhalo):
    # Function that returns Hernquist mass profile
    # inputs:
        # r = distance from galaxy (kpc)
        # a = scale radius (kpc)
        # Mhalo = total dark matter halo mass (10^12 Msun)
    # return:
        # total dark matter enclosed within r (Msun)
        return np.around(Mhalo*r**2/(a+r)**2,2)*u.Msun

    def CircularVelocity(self, PType, r):
    # Function that finds the circular speeds in (km/s), rounded to 2 decimal
    # places
    # inputs:
        # PType = particle type
        # array with radii
    # return:
        # array of circular speeds
        Menc = self.MassEnclosed(PType,r) # mass of each particle type
        CircVel = np.sqrt(G*Menc / r*u.kpc) # circ velocity of each particle
        return np.around(CircVel,2)

    def CircularVelocityTotal(self, r):
    # Function that finds the total circular velocity created by each galaxy
    # component at each radius of the radii array
    # input:
        # array of radii
    # return:
        # array of circular velocity
        M = self.MassEnclosedTotal(r) # total particle mass within each radius
        CircVel2 = np.sqrt(G*M / r*u.kpc) # total circ vel of the particles
        return np.around(CircVel2,2)

    def HernquistVCirc(self, r, a, Mhalo):
    # Function that finds circular speed (km/s) within the Hernquist M profile
    # inputs:
        # r = distance from galaxy (kpc)
        # a = scale radius (kpc)
        # Mhalo = total dark matter halo mass (10^12 Msun)
    # return:
        # circular speed (km/s) of mass in Hernquist M profile
        M2 = self.HernquistMass(r, a, Mhalo)
        CircVel3 = np.sqrt(G*M2 / r*u.kpc)
        return np.around(CircVel3,2)

In [3]:
# Defining the mass profile for each galaxy
MWMP = MassProfile("MW",0)
M31MP = MassProfile("M31",0)
M33MP = MassProfile("M33",0)

RR = 30 # test radius
testR = np.arange(0.1,RR+1,1.0)

In [5]:
# All the Milky Way values
ScaleMW = 61 # test scale radius
MtotMW = 1.975e12 # total Halo mass from M31
MtestMW = MWMP.MassEnclosed(1,testR) # mass enclosed from simulation
# Mass enclosed from Herquist profile
HtestMW = np.around(MWMP.HernquistMass(RR,ScaleMW,MtotMW)/1e10,2)
# Circular velocity using mass enclose from simulation
VtestMW = MWMP.CircularVelocity(1,testR)
# Circular velocity from Hernquist profile
HVtestMW = MWMP.HernquistVCirc(RR,ScaleMW,MtotMW)
print("Testing MW at 30 kpc")
print("********************")
print("Scale Radius =", ScaleMW)
print("Enclosed mass is",np.round(MtestMW[RR]/1e10),"x 1e10")
print("Hernquist =", HtestMW, "x 1e10")
print(" ")
print("Velocity =",VtestMW[RR])
print("Hernquist Vc =",HVtestMW)

Testing MW at 30 kpc
********************
Scale Radius = 61
Enclosed mass is 21.0 solMass x 1e10
Hernquist = 21.46 solMass x 1e10
 
Velocity = 173.78 km kpc / s
Hernquist Vc = 175.42 km kpc / s


In [6]:
# All the M31 values
ScaleM31 = 62 # test scale radius
MtotM31 = 1.921e12 # total Halo mass of M31 from HW3
MtestM31 = M31MP.MassEnclosed(1,testR) # mass enclosed from simulation
# Mass enclosed from Herquist profile
HtestM31 = np.around(M31MP.HernquistMass(RR,ScaleM31,MtotM31)/1e10,2)
# Circular velocity using mass enclose from simulation
VtestM31 = M31MP.CircularVelocity(1,testR)
# Circular velocity from Hernquist profile
HVtestM31 = M31MP.HernquistVCirc(RR,ScaleM31,MtotM31)
print("Testing M31 at 30 kpc")
print("********************")
print("Scale Radius =", ScaleM31)
print("Enclosed mass is",np.round(MtestM31[RR]/1e10),"x 1e10")
print("Hernquist =", HtestM31, "x 1e10")
print(" ")
print("Velocity =", VtestM31[RR])
print("Hernquist Vc =",HVtestM31)

Testing M31 at 30 kpc
********************
Scale Radius = 62
Enclosed mass is 21.0 solMass x 1e10
Hernquist = 20.43 solMass x 1e10
 
Velocity = 171.39 km kpc / s
Hernquist Vc = 171.13 km kpc / s


In [7]:
# All the M33 values
ScaleM33 = 25 # test scale radius
MtotM33 = 0.187e12 # total Halo mass of M33 from HW3
MtestM33 = M33MP.MassEnclosed(1,testR) # mass enclosed from simulation
# Mass enclosed from Herquist profile
HtestM33 = np.around(M33MP.HernquistMass(RR,ScaleM33,MtotM33)/1e10,2)
# Circular velocity using mass enclose from simulation
VtestM33 = M33MP.CircularVelocity(1,testR)
# Circular velocity from Hernquist profile
HVtestM33 = M33MP.HernquistVCirc(RR,ScaleM33,MtotM33)
print("Testing M33 at 30 kpc")
print("********************")
print("Scale Radius =", ScaleM33)
print("Enclosed mass is",np.round(MtestM33[RR]/1e10),"x 1e10")
print("Hernquist =", HtestM33, "x 1e10")
print(" ")
print("Velocity =", VtestM33[RR])
print("Hernquist Vc =",HVtestM33)

Testing M33 at 30 kpc
********************
Scale Radius = 25
Enclosed mass is 6.0 solMass x 1e10
Hernquist = 5.56 solMass x 1e10
 
Velocity = 89.53 km kpc / s
Hernquist Vc = 89.31 km kpc / s


In [ ]:
# The plots for all three Mass profiles
R = np.arange(0.1,31,0.5)

# For MW
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111)

# Plot mass enclosed for Halo 
plt.semilogy(R,MWMP.MassEnclosed(1,R), color='blue', linewidth=5, label='Halo')

# for Disk
plt.semilogy(R,MWMP.MassEnclosed(2,R), color='red', linestyle=":", linewidth=3, label='Disk')

# for bulge
plt.semilogy(R,MWMP.MassEnclosed(3,R), color='green', linestyle="-.",linewidth=3, label='Bulge')

# TOTAL Mass
plt.semilogy(R,MWMP.MassEnclosedTotal(R), color='black', linestyle="--",linewidth=5, label='Total')

# Hernquist Profile
plt.semilogy(R,MWMP.HernquistMass(R,ScaleMW,MtotMW), color='magenta',linewidth=5, label='Hernquist, a=61 kpc')

# Add axis labels
plt.xlabel('Radius (kpc)', fontsize=22)
plt.ylabel(r'Log(Mass Enclosed (M$_\odot$))', fontsize=22)

#set axis limits
plt.ylim(1e9,1e12)

#adjust tick label font size
label_size = 22
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size
# do it this way!!: 
#plt.rcParams.update[{'font.size':20}]


# add a legend with some customizations.
legend = ax.legend(loc='lower right',fontsize='x-large')

#add figure text
plt.figtext(0.15, 0.83, 'Milky Way Mass Profile', fontsize=22)


# Save to a file
ax.set_rasterized(True)
plt.savefig('MassProfile_MW.eps', rasterized=True, dpi=350)